In [ ]:
# To enable cost analysis of our sorting, we will have global counters
# for the number of swap operations performed, and the number of
# comparisons performed.
#
# Call resetCounts() before performing a sort
def resetCounts():
    global swapCount
    swapCount = 0
    global compareCount
    compareCount = 0
    
# Call reportCounts() after performing a sort to see what the cost was    
def reportCounts():
    global swapCount
    global compareCount
    return str(swapCount) + " swaps, "+ str(compareCount) + " compares"

# Call returnCounts() after performing a sort to get the cost
def returnCounts():
    global swapCount
    global compareCount
    return [swapCount, compareCount]

# This function swaps the two items at locations i and j in array A.
# (it also logs the usage for later reporting)
def swap(A, i, j):
    # Uncomment to see details of swaps:
    #print("swapping " + str(A[i]) + " with " + str(A[j]))
    temp = A[i]
    A[i] = A[j]
    A[j] = temp
    global swapCount
    swapCount += 1

# This function compares the two items at locations i and j in array A.
# It returns true if A[i] is greater than A[j] (the items must be
# comparable with the > operator).
# (The function also logs the usage for later reporting)
def compare(A, i, j):
    # Uncomment to see details of comparisons:
    #print("comparing " + str(A[i]) + " with " + str(A[j]))
    global compareCount
    compareCount += 1
    return A[i] > A[j]

In [ ]:
import math
import random

# Sort the items from i to j (inclusive) in A
def quickSort(A, i, j, method, verbose):
    # If there are fewer than 2 items to sort, then the items are
    # de-facto sorted
    if j<=i: return

    # Select the pivot item
    pivotIndex = 0
    if method == "a":
        # Use item i as the pivot
        pivotIndex = i;
    elif method == "b":
        # Use the midpoint as the pivot
        pivotIndex = i + math.floor((j-i)/2)
    elif method == "c":
        # Use a randomly selected item as the pivot
        pivotIndex = random.randint(i,j)
    
    if pivotIndex != i: swap(A, i, pivotIndex)
    
    # Partition the items from i+1 to j into two sets:
    # items less than or equal to the pivot, and
    # items greater than the pivot.
    
    # During processing, the array will look like this:
    # 1: | pivot | <=pivot || unprocessed | >pivot |
    # or this:
    # 2: | pivot | <=pivot | g | unprocessed || >pivot |
    # or this:
    # 3: | pivot | <=pivot | g | unprocessed | l | >pivot |
    #
    # Where pivot is the single pivot,
    # <=pivot is the items that have been found to be <= the pivot,
    # g is a single item that has been found to be greater than the pivot,
    # unprocessed is items that have not yet been compared to the pivot,
    # l is a single item that has been found to be less than the pivot,
    # >pivot is the items that have been found to be > the pivot
    #
    # We start in state 1 (with <=pivot and >pivot empty), and advance by
    # moving the || separator through the unprocessed items from the left
    # until we encounter an item that is > than the pivot.
    # We then move to state 2, and advance by moving the || separator
    # through the unprocessed items from the right until we encounter
    # an item that is <= the pivot.
    # We are then in state 3.  We swap g and l so they are both in the
    # correct partitions, and return to state 1.
    #
    # (We stop when unprocessed is empty.)

    # Keep track of the left and right ends of the unprocessed region
    left = i+1
    right = j
    
    while left <= right:
        # We're in state 1.  Advance through the unprocessed region from the
        # left until we find an item > than pivot
        while left <= right and not(compare(A, left, i)):
            left += 1
        # left is now the index of g (an item that is > pivot).
        # (Or left > right, meaning that we have finished the partition - 
        # either way, left is one spot to the right of the last <=pivot item
        # found so far).
            
        # We're in state 2.  Advance through the unprocessed region from the
        # right until we find an item <= pivot
        while left < right and compare(A, right, i):
            right -= 1
        # If left is equal to right, then we have finished the partition).
        if left == right: right -= 1
        # right is now the index of l (or right is < left - either way,
        # right is one spot to the left of the first >pivot item found so far).
        
        # We're in state 3. (Or we're done)
        if left < right:
            swap(A, left, right)
            left += 1
            right -= 1
        
        # Go back to state 1
        
    # Partition complete. left is the index above the last item that is <= pivot.
    # Put the pivot in between the <=pivot and >pivot.
    if left != i+1: swap(A, i, left-1)
    
    # Uncomment to print the intermediate results
    if verbose: print(A)
    
    # The pivot is now in its final sorted position. So it's time to recurse!
    # Sort everything to the left of the pivot.
    quickSort(A, i, left-2, method, verbose)
    # Sort everything to the right of the pivot.
    quickSort(A, left, j, method, verbose)

In [ ]:
# Example of quickSort in action:
Eg = list(range(1,10))
random.Random(1).shuffle(Eg)

resetCounts()
print("starting:")
print(Eg)
print("")

quickSort(Eg, 0, len(Eg)-1, "a", True)

print("sorted:")
print(Eg)
reportCounts()

In [ ]:
# A function to test the cost of the quickSort function by running it lots of times (reps).
# Use the method ("a", "b", or "c").
# Run quickSort on the array A, either keeping A the same every time, or
# shuffling it randomly every time (according to shuffle).
# Reports:
# best (lowest) number of item-item comparisons
# worst (highest) number of item-item comparisons
# average (per sort) number of item-item comparisons
def quickSortTester(method, A, reps, shuffle):
    best = float('inf')
    worst = -1;
    average = 0;
    
    for i in range(0, reps):
        # Make a copy of A
        Acopy = A.copy()
        # Shuffle it if called for
        if shuffle: random.shuffle(Acopy)
        
        # Run the sort
        resetCounts()
        quickSort(Acopy, 0, len(Acopy)-1, method, False)
        
        compares = returnCounts()[1]
        if compares < best: best = compares
        if compares > worst: worst = compares
        average += compares/reps
        
    return [worst, best, average]

In [ ]:
# Part (a):
# Create an ordering of the integers from 1 to 7 that results
# in the worst-case performance of quickSort (using method "a" - i.e., using
# the first item in the list as the pivot).
# You can write a function to do this if you want.  In the end, call the
# list aWorstCase7.
aWorstCase7 = [3,0,1,8,7,2,5,4,6,9]

# Also create an ordering of the integers from 1 to 15 that results
# in the worst-case performance. You can write a function to do this
# if you want.  In the end, call the list aWorstCase15
aWorstCase15 = [3,0,1,8,7,2,5,4,6,9]

# Do the same for 1-31, 1-63, 1-127, and 1-255
aWorstCase31 = [3,0,1,8,7,2,5,4,6,9]
aWorstCase63 = [3,0,1,8,7,2,5,4,6,9]
aWorstCase127 = [3,0,1,8,7,2,5,4,6,9]
aWorstCase255 = [3,0,1,8,7,2,5,4,6,9]

In [ ]:
# Test quickSort on the worst-case inputs
result = quickSortTester("a", aWorstCase7, 1, False)
print(str(result[0]) + " compares on aWorstCase7")
result = quickSortTester("a", aWorstCase15, 1, False)
print(str(result[0]) + " compares on aWorstCase15")
result = quickSortTester("a", aWorstCase31, 1, False)
print(str(result[0]) + " compares on aWorstCase31")
result = quickSortTester("a", aWorstCase63, 1, False)
print(str(result[0]) + " compares on aWorstCase63")
result = quickSortTester("a", aWorstCase127, 1, False)
print(str(result[0]) + " compares on aWorstCase127")
result = quickSortTester("a", aWorstCase255, 1, False)
print(str(result[0]) + " compares on aWorstCase255")

In [ ]:
# Part (b): Choosing the middle item in the array to be the pivot should
# help with the worst case.
result = quickSortTester("b", aWorstCase7, 1, False)
print(str(result[0]) + " compares on aWorstCase7")
result = quickSortTester("b", aWorstCase15, 1, False)
print(str(result[0]) + " compares on aWorstCase15")
result = quickSortTester("b", aWorstCase31, 1, False)
print(str(result[0]) + " compares on aWorstCase31")
result = quickSortTester("b", aWorstCase63, 1, False)
print(str(result[0]) + " compares on aWorstCase63")
result = quickSortTester("b", aWorstCase127, 1, False)
print(str(result[0]) + " compares on aWorstCase127")
result = quickSortTester("b", aWorstCase255, 1, False)
print(str(result[0]) + " compares on aWorstCase255")

In [ ]:
# ... but it is still possible for quickSort to have just as bad a worst-case
# when using the middle item as the pivot.
# Create orderings of the integers from 1-7, 1-15, 1-31, 1-63, 1-127 and 1-255
# that result in the worst-case performance of quickSort (using method "b" -
# i.e., using the middle item in the list as the pivot).
# You can write a function to do this if you want.  In the end, call the
# lists bWorstCase7, bWorstCase15, ..., bWorstCase255.
bWorstCase7= [3,0,1,8,7,2,5,4,6,9]
bWorstCase15 = [3,0,1,8,7,2,5,4,6,9]
bWorstCase31 = [3,0,1,8,7,2,5,4,6,9]
bWorstCase63 = [3,0,1,8,7,2,5,4,6,9]
bWorstCase127 = [3,0,1,8,7,2,5,4,6,9]
bWorstCase255 = [3,0,1,8,7,2,5,4,6,9]


In [ ]:
# Test quickSort on the b worst-case inputs
result = quickSortTester("b", bWorstCase7, 1, False)
print(str(result[0]) + " compares on bWorstCase7")
result = quickSortTester("b", bWorstCase15, 1, False)
print(str(result[0]) + " compares on bWorstCase15")
result = quickSortTester("b", bWorstCase31, 1, False)
print(str(result[0]) + " compares on bWorstCase31")
result = quickSortTester("b", bWorstCase63, 1, False)
print(str(result[0]) + " compares on bWorstCase63")
result = quickSortTester("b", bWorstCase127, 1, False)
print(str(result[0]) + " compares on bWorstCase127")
result = quickSortTester("b", bWorstCase255, 1, False)
print(str(result[0]) + " compares on bWorstCase255")

In [ ]:
# Part (c): From parts (a) and (b), we can tell that any implementation of
# Quick Sort will always have some worst-case input that will result in a
# worst-case run time that is big-O worse than the worst-case performance of
# Merge Sort.
# But _randomly_ selecting the pivot from all available items helps make it
# very unlikely that the algorithm will encounter that worst-case input
# (or close-to-worst-case inputs).
#
# Let's see how randomized Quick Sort does on the worst-case inputs from
# (a) and (b)
result = quickSortTester("c", aWorstCase7, 5000, False)
print("aWorstCase7")
print("Worst case: " + str(result[0]) + " compares")
print("Best case: " + str(result[1]) + " compares")
print("Average: " + str(result[2]) + " compares")
print("")

result = quickSortTester("c", bWorstCase7, 5000, False)
print("bWorstCase7")
print("Worst case: " + str(result[0]) + " compares")
print("Best case: " + str(result[1]) + " compares")
print("Average: " + str(result[2]) + " compares")
print("")